<a href="https://colab.research.google.com/github/CameronKenworthyCode/NEWICC/blob/main/PLSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymongo[srv]
!pip install dnspython
!pip install requests
!pip install bs4
!pip install html2text
!pip install -U -q PyDrive

import html2text
from ast import literal_eval
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
import numpy as np
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import random
!pip install selenium
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

auth.authenticate_user() #establish connection to a gdrive to write the eventual csv file to
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1DWFETkLt-dWcA7CfymejJe7O-UJxSR4B' in parents and trashed=False"}).GetList() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 8.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.10
    Uninstalling urllib3-1.26.10:
      Successfully uninstalled urllib3-1.26.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.3.0 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.10-py2.py3-none-any.whl (139 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Mounted at /content/drive/


In [5]:
file_list[0]

GoogleDriveFile({'kind': 'drive#file', 'id': '1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1', 'etag': '"MTY1ODA5MzQzMzMzNw"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1', 'webContentLink': 'https://drive.google.com/uc?id=1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1&export=download', 'alternateLink': 'https://colab.research.google.com/drive/1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1', 'embedLink': 'https://drive.google.com/file/d/1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/vnd.google.colaboratory', 'title': 'PLSS.ipynb', 'mimeType': 'application/vnd.google.colaboratory', 'description': 'Colaboratory notebook', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2022-07-16T23:41:41.265Z', 'modifiedDate': '2022-07-17T21:30:33.337Z', 'modifiedByMeDate': '2022-07-17T21:30:33.337Z', 

# TODO

- generate URLS
- batch query https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs=
- save geometry long/lat coords
  - number of coords given varies per section
    - get corners
      - get all points along each edge
    - infer outline of township range from edges of sections
- infer shape from coords
- save all as csv
- code to calculate if coord is in shape

#Generate URL Queries

TOWN: 1N -> 41N

RANGE: 16W -> 47E


In [6]:
# generate set of strings in form WA330##0N0##0E0SN##0 or WA330##0N0##0W0SN##0
def generate_url(town, _range):
  t_ = town[:-1]
  if int(t_) < 10:
    t_ = '0' + t_
  r_ = _range[:-1]
  if int(r_) < 10:
    r_ = '0' + r_
  r_ = r_ + '0' + _range[-1:]
  base_string = 'WA330'+ t_ + '0N0' + r_ + '0SN'
  section_list = []
  for i in range(1, 60):
    s_ = str(i) + '0'
    if i < 10:
      s_ = '0' + str(i) + '0'
    section_list.append(base_string+s_)
  
  return section_list

In [ ]:
def trs_from_url(url_string):
  

In [51]:
trs = []

for i in range(1,42):
  for j in range(1,17):
    trs.append([str(i)+'N', str(j)+'W'])

for i in range(1, 42):
  for j in range(1, 48):
    trs.append([str(i)+'N', str(j)+'E'])

url_dict = {}
for pair in trs:
  key = pair[0] + '-' + pair[1]
  value = generate_url(pair[0], pair[1])
  url_dict[key] = value

# go through each T/R pair, and take the next 25 section urls to build a query, save the results and continue to the next 25
PLSS_dict = {}
base_url = 'https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs='
built_url = ''
base_url_end = '&returnalllevels=&f=pjson'
count = 0
keys = {}
urls = []
for i in range(len(trs)):
  for j in range(0, 59):
    if count == 25: 
      urls.append(base_url+built_url+base_url_end)
      #print(base_url+built_url+base_url_end)
      #query_result = query(base_url+build_url+base_url_end) # send query
      #PLSS_dict[trs[i][0] + '-' + trs[i][1] + '-' + j] = query_result
      built_url = ''
      count = 0
    count += 1
    built_url = built_url + url_dict[trs[i][0] + '-' + trs[i][1]][j] + '+%7C+'
    keys[url_dict[trs[i][0] + '-' + trs[i][1]][j]] = trs[i][0] + '-' + trs[i][1] + '-' + str(j+1) 

In [45]:
import json

In [105]:
parser = html2text.HTML2Text()
parser.ignore_links = True
result = requests.get(urls[0])
src = result.content
soup = BeautifulSoup(src, 'lxml')

In [104]:
for url in urls:
  parser = html2text.HTML2Text()
  parser.ignore_links = True
  result = requests.get(url)
  src = result.content
  soup = BeautifulSoup(src, 'lxml')
  result = json.loads(soup.text)
  for i in range(len())

KeyboardInterrupt: ignored

In [106]:
result = json.loads(soup.text)

In [107]:
result['features']

{'coordinates': [],
 'features': [],
 'generatedplss': ['WA330010N0010W0SN010',
  'WA330010N0010W0SN020',
  'WA330010N0010W0SN030',
  'WA330010N0010W0SN040',
  'WA330010N0010W0SN050',
  'WA330010N0010W0SN060',
  'WA330010N0010W0SN070',
  'WA330010N0010W0SN080',
  'WA330010N0010W0SN090',
  'WA330010N0010W0SN100',
  'WA330010N0010W0SN110',
  'WA330010N0010W0SN120',
  'WA330010N0010W0SN130',
  'WA330010N0010W0SN140',
  'WA330010N0010W0SN150',
  'WA330010N0010W0SN160',
  'WA330010N0010W0SN170',
  'WA330010N0010W0SN180',
  'WA330010N0010W0SN190',
  'WA330010N0010W0SN200',
  'WA330010N0010W0SN210',
  'WA330010N0010W0SN220',
  'WA330010N0010W0SN230',
  'WA330010N0010W0SN240',
  'WA330010N0010W0SN250'],
 'status': 'fail',
 'statusmsg': 'No results returned.',
 'trs': 'WA330010N0010W0SN010 | WA330010N0010W0SN020 | WA330010N0010W0SN030 | WA330010N0010W0SN040 | WA330010N0010W0SN050 | WA330010N0010W0SN060 | WA330010N0010W0SN070 | WA330010N0010W0SN080 | WA330010N0010W0SN090 | WA330010N0010W0SN100 |

In [99]:
result['features'][0]['geometry']['rings'][0]

[[-122.82433815198667, 46.532295862909734],
 [-122.8243487224626, 46.52867750588515],
 [-122.82435922107337, 46.525059148855476],
 [-122.82436836322798, 46.521428219395546],
 [-122.82437743351741, 46.51779721911253],
 [-122.82964107778874, 46.517797790907764],
 [-122.83490465019483, 46.517798004171944],
 [-122.84013572245225, 46.51781386051464],
 [-122.84536672284447, 46.5178295029705],
 [-122.84535993786912, 46.521456574209324],
 [-122.84535315379212, 46.525083717306686],
 [-122.84534643978368, 46.52871078943568],
 [-122.84533965570665, 46.53233793270174],
 [-122.84008972511646, 46.532327790223256],
 [-122.8348398672898, 46.532317433917726],
 [-122.82958900963823, 46.53230679147991],
 [-122.82433815198667, 46.532295862909734]]

In [94]:
for i in range(len(result['features'])):
  print(keys[result['features'][i]['attributes']['landdescription']])

12N-1W-17
12N-1W-18
12N-1W-19
12N-1W-20
12N-1W-20
12N-1W-21
12N-1W-22
12N-1W-23
12N-1W-24
12N-1W-25
12N-1W-26
12N-1W-27
12N-1W-28
12N-1W-29
12N-1W-30
12N-1W-31
12N-1W-32
12N-1W-33
12N-1W-33
12N-1W-34
12N-1W-35
12N-1W-35
12N-1W-35
12N-1W-36


In [29]:
print(urls[416])

https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs=WA330120N0010W0SN170+%7C+WA330120N0010W0SN180+%7C+WA330120N0010W0SN190+%7C+WA330120N0010W0SN200+%7C+WA330120N0010W0SN210+%7C+WA330120N0010W0SN220+%7C+WA330120N0010W0SN230+%7C+WA330120N0010W0SN240+%7C+WA330120N0010W0SN250+%7C+WA330120N0010W0SN260+%7C+WA330120N0010W0SN270+%7C+WA330120N0010W0SN280+%7C+WA330120N0010W0SN290+%7C+WA330120N0010W0SN300+%7C+WA330120N0010W0SN310+%7C+WA330120N0010W0SN320+%7C+WA330120N0010W0SN330+%7C+WA330120N0010W0SN340+%7C+WA330120N0010W0SN350+%7C+WA330120N0010W0SN360+%7C+WA330120N0010W0SN370+%7C+WA330120N0010W0SN380+%7C+WA330120N0010W0SN390+%7C+WA330120N0010W0SN400+%7C+WA330120N0010W0SN410+%7C+&returnalllevels=&f=pjson


In [ ]:
built_url = ''
built_url = built_url + url_dict[trs[0][0] + '-' + trs[0][1]][0] + '+%7C+'

In [ ]:
url_dict[trs[0][0] + '-' + trs[0][1]]

https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs=

WA33 0**TT**0N 0**RR**0E 0SN**SS**0

25 TRS per query

```
WA330270N0310E0SN060+%7C+ # 27/31/06
WA330160N0260E0SN120+%7C+
WA330170N0300E0SN170+%7C+
WA330170N0280E0SN300+%7C+
WA330160N0240E0SN130+%7C+
WA330160N0250E0SN080+%7C+
WA330160N0240E0SN130+%7C+
WA330200N0280E0SN020+%7C+
WA330210N0280E0SN340+%7C+
WA330190N0250E0SN040+%7C+
WA330200N0230E0SN040+%7C+
WA330200N0280E0SN020+%7C+
WA330210N0280E0SN340+%7C+
WA330170N0300E0SN080+%7C+
WA330180N0300E0SN240+%7C+
WA330180N0300E0SN260+%7C+
WA330200N0280E0SN020+%7C+
WA330200N0280E0SN020+%7C+
WA330190N0290E0SN280+%7C+
WA330190N0290E0SN280+%7C+
WA330190N0290E0SN330+%7C+
WA330170N0240E0SN100+%7C+
WA330170N0240E0SN110+%7C+
WA330190N0270E0SN080+%7C+
WA330200N0250E0SN110+%7C+
```



&returnalllevels=&f=pjson&__ncforminfo=7x7RLQJqpDQ92T4PQI_-wSk34fhSp0krCuCkP5nOMe9rVpPf2stN7FAL0CQsbN_c62SdT15Xs7Rs5U4Xfe3epxz0dE12hq0HyWAsrAz430mfnjihcVff8g%3D%3D

In [ ]:
#